In [1]:
# import libraries
import sys, os, signal, time, threading, json, webbrowser

import jupyros
import rospy, message_filters, actionlib
from std_msgs.msg import String, Header
from actionlib_msgs.msg import GoalStatus

! source /home/qcr/cgras_moveit_ws/devel/setup.bash
sys.path.append('/home/qcr/cgras_moveit_ws/devel/lib/python3/dist-packages')

# project modules
from cgras_robot.msg import RobotCommandAction, RobotCommandGoal

rospy.init_node('action_node')

In [2]:
jupyros.action_client('/cgras/robot/do', RobotCommandAction, RobotCommandGoal, callbacks={})

[INFO] [1704351603.877784]: [/CGRAS/ROBOT/DO] Waiting for action server.
[INFO] [1704351603.900518]: [/CGRAS/ROBOT/DO] Connection to server successful.


In [3]:
def cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print(f'success: the location apriltag found {result.data}')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')   

def cb_move_done(status, result):   
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the target')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')   

def cb_reset_done(status, result): 
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the desired pose')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.error}')             

# -- callback from the GUI console
def call_robot(event, *args):
    if event == 'calibrate':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
        action_client.wait_for_server()
        goal = RobotCommandGoal(target=[RobotCommandGoal.ACTION_CALIBRATE])
        action_client.send_goal(goal, done_cb=cb_calibrate_done)
            
    elif event == 'move_cell':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
        action_client.wait_for_server()
        if len(args) == 0:
            target = [RobotCommandGoal.ACTION_MOVE_CELL, 0, 0, 0, 0]
        else:
            target = [RobotCommandGoal.ACTION_MOVE_CELL] + args[0]
        goal = RobotCommandGoal(target=target)
        action_client.send_goal(goal, done_cb=cb_move_done)

    elif event == 'move_name':
        action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
        action_client.wait_for_server()
        if len(args) == 0:
            target = [RobotCommandGoal.ACTION_MOVE_NAME, RobotCommandGoal.POSENAME_STOW]
        else:
            target = [RobotCommandGoal.ACTION_MOVE_CELL] + args
        goal = RobotCommandGoal(target=target)
        action_client.send_goal(goal, done_cb=cb_reset_done)

# call_robot('reset')
call_robot('move_cell', [0, 0, 0, 0,])

success: moved to the target
